# Enact Concepts



## Resources

Enact is a framework for generative software, which offers the ability to track
python programs and their executions. To this end, any tracked object in the
enact framework is an instance of a `Resource`, including data, executable
components and journaled executions.

New resources can be defined and registered with the enact framework as follows:

In [1]:
import enact

import dataclasses

@enact.register
@dataclasses.dataclass
class MyResource(enact.Resource):
  my_field: str
  my_other_field: int

A `Resource` subclass defines a unique type ID and an interface to access field
names and their values:

In [2]:
print(f'TypeID of MyResource: {repr(MyResource.type_id())}')
print(f'Fields of my_resource: {list(MyResource.field_names())}')

my_resource = MyResource('hello', 42)
print(f'Value of my_field: {my_resource.my_field}')
print(f'Value of my_other_field: {my_resource.my_other_field}')
print(f'Field values of the resource: {list(my_resource.field_values())}')
print(f'Field items: {list(my_resource.field_items())}')

TypeID of MyResource: '{"digest": "a46cd7bf341ef134fe8fd7e97e564f0f84425c747c3c55850cd599f1249b97f5", "name": "__main__.MyResource"}'
Fields of my_resource: ['my_field', 'my_other_field']
Value of my_field: hello
Value of my_other_field: 42
Field values of the resource: ['hello', 42]
Field items: [('my_field', 'hello'), ('my_other_field', 42)]


Resources may be nested and they can contain maps and lists, but the
leaf types are limited to:
* Python primitives of type `int`, `float`, `bool`, `str`, `bytes`, `bool`,
* the `None` value,
* other resources (i.e., instances of `ResourceBase`),
* types of resources (i.e., subclasses of `ResourceBase`).

Python maps and lists are allowed and may be nested, but map keys must be of type `str`.

In [3]:
from typing import Dict, List, Union, Type

@dataclasses.dataclass
class ComplexResource(enact.Resource):
  primitives: List[Union[int, float, bool, str, bytes, bool, type(None)]]
  map_value: Dict[str, MyResource]
  nested_resource: MyResource
  resource_type: Type[MyResource]

ComplexResource(
  primitives=[1, 2.0, False, None, 'test', b'bytes'],
  map_value={'hello': MyResource('hello', 69)},
  nested_resource=MyResource('hola', 42),
  resource_type=MyResource).to_resource_dict()

{'primitives': [1, 2.0, False, None, 'test', b'bytes'],
 'map_value': {'hello': {'my_field': 'hello', 'my_other_field': 69}},
 'nested_resource': {'my_field': 'hola', 'my_other_field': 42},
 'resource_type': __main__.MyResource}

## Stores and References

`Resource` instances can be committed to a store which yields an
immutable reference to the instance in its current state. References
can be checked out of the store.

In [4]:
with enact.InMemoryStore() as store:
  # Enter the context of the store.
  my_resource = MyResource('hello', 42)
  ref = enact.commit(my_resource)  # commits the resource to the active store.
  print(ref)
  print(ref.checkout())

<Ref: 947051f773d0e01ed76be7506fe4cfde5be54d35eef91f10b488d3e8b789ee3b>
MyResource(my_field='hello', my_other_field=42)


Since checking out references is a common operation, `ref.checkout()`
may be abbreviated to simply `ref()`.

In [5]:
with store:
  print(ref())

MyResource(my_field='hello', my_other_field=42)


References point to resources, but they are also resources themselves. This
means that resources can have fields that reference other resources.

In [6]:
from typing import Optional


@enact.register
@dataclasses.dataclass
class LinkedList(enact.Resource):
  """A linked list of resources."""
  value: enact.FieldValue
  next: Optional[enact.Ref['LinkedList']] = None

with store:
  # Create a linked list with two nodes.
  l = enact.commit(LinkedList(value=my_resource))
  l = enact.commit(LinkedList(value=MyResource('hola', 69), next=l))
  # Iterate through the list.
  while l != None:
    print(l().value)
    l = l().next

MyResource(my_field='hola', my_other_field=69)
MyResource(my_field='hello', my_other_field=42)


## Invokables

Invokables are executable resources that allow for journaled execution which can
be rewound and replayed. They accept a resource-typed argument and produce a
resource-typed output. Invokables may be annotated with their specific input and
output types using the `typed_invokable` decorator, which will also register the
resource with the enact framework.

In [7]:
import random


@enact.typed_invokable(type(None), int)
@dataclasses.dataclass
class RollDie(enact.Invokable[None, int]):
  sides: int = 6
  def call(self) -> int:
    return random.randint(1, self.sides)

@enact.typed_invokable(int, int)
@dataclasses.dataclass
class RollDice(enact.Invokable):
  die: enact.Ref[enact.Invokable[None, int]]
  def call(self, num_rolls: int) -> int:
    die = self.die.checkout()
    return sum(die() for _ in range(num_rolls))
  

Invokables can be called just like any python callable.

In [8]:
with store:
  # Create the die to use.
  die = enact.commit(RollDie())
  # Create invokable that rolls the die repeatedly.
  dice = RollDice(die)
  # Roll the die thrice and report sum.
  print(dice(3))

14


Additionally they support journaled execution using the `invoke` function, which
recursively tracks inputs, outputs and raised exceptions.

In [9]:
with store:
  num_rolls = enact.commit(3)
  invocation = dice.invoke(num_rolls)
  
def print_rolls(invocation):
  print(f'Individual dice Rolls:')
  for i, child in enumerate(invocation.get_children()):
    print(f'  roll {i}: {child.get_output()}')
  print(f'Output is: {enact.pformat(invocation.response().output)}')
  
print_rolls(invocation)

Individual dice Rolls:
  roll 0: 1
  roll 1: 4
  roll 2: 2
Output is: -> 7#fbf3c2


## Advanced concepts

### Storage model

When committing a resource, the returned reference refers to a cryptographic
digest of the resource type and it's content. 

This means that when committing an identical resource instance twice, the same
reference is returned:

In [10]:
with store:
  r1 = enact.commit(MyResource('hello', 42))
  r2 = enact.commit(MyResource('hello', 42))
  print(r1 == r2)

True


Conceptually, stores can be viewed as hashtables which, due to their use of long
hash digests, have very low collision probability. This means that separate enact
stores containing different data, can be interpreted as providing partial views 
on the same global 'address space'.

Stores contain resources that may contain references to other resources. They
can therefore be interpreted as directed graphs. For example, invocations
point at the input and output resources.

Note that it is not possible to commit two resources that reference each other:
Taking a reference requires creating a cryptographic digest of resource
contents, including all of its own reference fields. Therefore, to compute the
digest of the first resource, we would need to first compute the digest of the
second - but this would in turn require computing the digest of the first:

In [11]:
with store:
  l1 = LinkedList(value=1, next=None)
  l2 = LinkedList(value=2, next=enact.commit(l1))

  # This does not create a cycle, since l2 points at the previously committed
  # version of l1.
  l1.next = enact.commit(l2)

  # We now have list of length 3 instead of a cycle:
  enact.pprint(enact.commit(l1))

-> LinkedList#0b182f:
  value:: 1
  next:
    -> LinkedList#0f9059:
      value:: 2
      next:: -> LinkedList(value=1, next=None)#dd2ab0


Notice that the digest (after the `#` symbol) is different for the three linked
list nodes.

Since resources can't reference each other store graphs are acyclic, which means
that they form a _directed acyclic graph_ (DAG). DAGs that employ this form of
hashing are known as Merkle DAGs or hash DAGs. Hash DAGs are generalizations of
data structures such as Merkle trees and block chains.

### Rewinding and replaying invocations

Invocations represent journaled executions, that store the
inputs and outputs of all intermediate invokables. Because of this,
we can use them to replay a previous execution. For example, in order
to reroll the last die in an dice roll execution we can call `rewind`
to remove a specified number of calls from the invocation object and then
`replay` the invocation, which will only reexecute the now missing calls.

In [12]:
with store:
  invocation = dice.invoke(enact.commit(3))
  print_rolls(invocation)

  invocation = invocation.rewind(2)
  print('\n==Rewound invocation to:')
  print_rolls(invocation)

  print('\n==Replayed invocation as:')
  invocation = invocation.replay()
  print_rolls(invocation)


Individual dice Rolls:
  roll 0: 3
  roll 1: 3
  roll 2: 6
Output is: -> 12#c231c9

==Rewound invocation to:
Individual dice Rolls:
  roll 0: 3
Output is: None

==Replayed invocation as:
Individual dice Rolls:
  roll 0: 3
  roll 1: 3
  roll 2: 1
Output is: -> 7#fbf3c2


Note that the `call` function of any invocation that raised an exception will
be reexecuted, although if it had subinvocations that succeeded, those will
be replayed.

Replays also support an `exception_override` argument which allows injecting
values instead of replaying an exception. This is the mechanism with which
[input requests](#input-requests) are handled:

In [13]:
class Halt(enact.ExceptionResource):\
  pass

@enact.typed_invokable(type(None), str)
class RaiseHalt(enact.Invokable):
  def call(self):
    raise Halt()

def exception_override(exception_ref):
  if isinstance(exception_ref(), Halt):
    return str('Injected value')

with store:
  invocation = RaiseHalt().invoke()
  assert isinstance(invocation.get_raised(), Halt)
  print(f'Output: {invocation.replay(exception_override).get_output()}')


Output: Injected value


#### Side note: Replays and non-determinism

Replays work by reexecuting invokables that have no known output in the 
replayed invocation. During reexecution, calls to child invokables must 
be matched to the recorded calls in order to see if an output can be replayed.
Therefore, for the replay functionality to work properly, the next call to a
subinvokable must be a deterministic function of the invokable input and the
return values of previous subinvokables.

That is, all sources of non-determinism must either be wrapped in invokable
objects or must make no difference to the call sequence. For example, for
subinvokables `x`, `y`, `z`:
1. `return random.choice([y, z])()`: Not replayable, since non-determinism
affects the call order.
2. `return x() ? y() : z()`: Replayable, since any non-determinism in `x` is
contained within an invokable.
3. `return enact.Int(x() + random.randint(0, 5))`: Replayable, since
non-determinism does not effect call order or call arguments to subinvokables.

The following example shows what happens when this property is violated.

In [14]:
import datetime
import time


@enact.typed_invokable(float, str)
class FormatTimestamp(enact.Invokable):
  """Formats a timestamp as a string."""

  def call(self, timestamp: float) -> str:
    return str(datetime.datetime.fromtimestamp(timestamp))

@enact.typed_invokable(type(None), str)
class FormatCurrentTime(enact.Invokable):
  """Return the current time as a string."""
  
  def call(self) -> str:
    now = time.time()
    # ERROR: Non-deterministic call argument to subinvokable:
    return FormatTimestamp()(now)

with store:
  invocation = FormatCurrentTime().invoke()
  # Delete only the output to force reexecution.
  invocation = invocation.rewind(num_calls=0)
  try:
    invocation.replay()
  except enact.ReplayError as e:
    print(f'Got error: {e}')

Got error: Expected invocation FormatTimestamp()(1694477751.8120074) but got FormatTimestamp()(1694477751.8082361).
Ensure that calls to subinvokable are deterministic or use strict=False.


To fix the above example, we could either wrap `time.time()` in an invokable
`CurrentTime` or, alternatively, define `FormatTimestamp` as a normal python
function instead of making it an invokable.

### Input Requests

Input requests are a mechanism for interrupting an ongoing execution in order to
collect external input from a user or system. (This is conceptually
similar to generators or continuations).

Input requests can be generated by calling the `request_input` function, which
takes as arguments a resource type that is requested, an optional resource for
which input is requested and a context for the request (e.g., instructions to 
a user).


In [15]:
@enact.typed_invokable(type(None), int)
class DiceRollWithUserConfirmation(enact.Invokable):
  def call(self, num_calls: int) -> int:
    roll_die = RollDie()
    total = 0
    for i in range(num_calls):
      while True:
        score = roll_die()
        if enact.request_input(
            requested_type=str,
            for_value=score,
            context=f'Should I reroll die #{i}?') == 'yes':
          continue
        break
      total += score
    return int(total)

The `InvocationGenerator` class provides a python `Generator` interface for
processing an invocation step by step, stopping at InputRequests.

In [16]:
with store:
  inv_gen = enact.InvocationGenerator(
    DiceRollWithUserConfirmation(),
    enact.commit(3))
  
  for input_request in inv_gen:
    roll = input_request.for_value()
    if roll < 5:
      inv_gen.set_input('yes')
    else:
      inv_gen.set_input('no')
  
  # Print invocation history:
  invocation = inv_gen.invocation
  print('Total score:', invocation.get_output())
  for child in invocation.get_children():
    output = child.get_output()
    if isinstance(output, int):
      print('  Die roll:', output)
    else:
      print('   -Rerolled?', output)

Total score: 18
  Die roll: 6
   -Rerolled? no
  Die roll: 6
   -Rerolled? no
  Die roll: 1
   -Rerolled? yes
  Die roll: 2
   -Rerolled? yes
  Die roll: 2
   -Rerolled? yes
  Die roll: 3
   -Rerolled? yes
  Die roll: 3
   -Rerolled? yes
  Die roll: 6
   -Rerolled? no


### Modifying references

While references point to immutable snapshots of resource objects, it is
possibly to change which resource a given `Ref` object points to using the
`modify()` context manager:

In [17]:
with store:
  ref = enact.commit(MyResource('hello', 42))
  ref_copy = ref.deepcopy_resource()
  with ref.modify() as my_resource:
    my_resource.my_other_field = 69
  print(f'ref points at new resource:\n  {ref()}')
  print(f'but the old resource is still intact:\n  {ref_copy()}')
  

ref points at new resource:
  MyResource(my_field='hello', my_other_field=69)
but the old resource is still intact:
  MyResource(my_field='hello', my_other_field=42)


### Custom store backends and reference types

A store is connected to a backend which defines how resources are actually
stored. Calling `InMemoryStore()` is equivalent to calling
`Store(InMemoryBackend())`, and similarly, `FileStore(root_dir)` is equivalent
to `Store(FileBackend(root_dir))`.

New storage backends can be created by implementing the following interface:

```python
class StorageBackend(abc.ABC):
  """A storage backend."""

  @abc.abstractmethod
  def commit(self, packed_resource: PackedResource):
    """Stores a packed resource."""

  @abc.abstractmethod
  def has(self, ref: Ref) -> bool:
    """Returns whether the storage backend has the resource."""

  @abc.abstractmethod
  def get(self, ref: Ref) -> Optional[interfaces.ResourceDict]:
    """Returns the packed resource or None if not available."""
```

Resources are _packed_ into serializable dictionaries before being committed.
This additional step can be overriden in custom `Ref` subclasses in order to
support advanced functionality such as compression or end-to-end encryption of
resources.


### Meta-invocations and higher-order functions.

Invokables support two core operations:
1. They can be _called_, like a normal python callable.
2. They can be _invoked_, which corresponds to a journaled execution.

Typically, invokables will _call_ other invokables, but they may also _invoke_
them, which is useful if an invokable wants to study the execution path of
another invokable.

This is useful, for example, when dealing with AI-generated code, since the
the execution trace of a generated function can be used as feedback to the
generation process.

We illustrate the concept of meta-invocation with a simple example using dice
wherein we create an invokable that analyzes another invokable's execution to
determine the number of execution steps.

In [18]:
@enact.typed_invokable(RollDie, int)
class RollUntilPrime(enact.Invokable):
  """Rolls a die until the sum of the rolls is prime."""
  def call(self, die: RollDie) -> int:
    total = 0
    def is_prime(n):
      return n > 1 and all(n % i for i in range(2, n))
    while not is_prime(total):
      total += die()
    return total

@enact.typed_invokable(enact.Request, enact.Invocation)
class Invoke(enact.Invokable):
  """Returns the result of invoking the given request."""
  def call(self, request: enact.Request) -> enact.Invocation:
    return request.invokable().invoke(request.input)

@enact.typed_invokable(RollDie, int)
class CountRolls(enact.Invokable):
  """Analyze the execution trace to see how long it takes to roll a prime."""
  def call(self, die: RollDie) -> int:
    invoke = Invoke()
    invocation = invoke(enact.Request(
      enact.commit(RollUntilPrime()), enact.commit(die)))
    return len(list(invocation.get_children()))

with store:
  count_rolls = CountRolls()
  invocation = count_rolls.invoke(enact.commit(RollDie(6)))
  print(f'Rolls until prime: {invocation.get_output()}')
  # Get output of call to invoke(...)
  meta_invocation = invocation.get_child(0).get_output()
  print(f'The prime that was rolled: {meta_invocation.get_output()}')


Rolls until prime: 2
The prime that was rolled: 2


### Support for asyncio

Python coroutines provide a model for concurrent execution of python code. This
is useful in cases where, e.g., a generative component needs to query multiple
APIs. Instead of waiting on the output of one call before executing the next,
both calls can be issued concurrently, e.g.:

```python
import asyncio

# Define a coroutine using the 'async' keyword.
async def wait_on_api_calls():
  # Use the 'await' keyword to suspend execution until both the api_call1 and
  # the api_call2 coroutines have completed.
  result = await asyncio.gather(api_call1(), api_call2())
  return result

def main():
  asyncio.run(wait_on_api_calls())  # Run the coroutine in an event loop.
```

For more information on `async`, see the [python documentation](https://docs.python.org/3.8/library/asyncio.html).

Enact supports asyncio components through the `AsyncInvokable` base class. 
Working with `AsyncInvokable` components is similar to working with `Invokable`
components, except that the `call` and `invoke` functions are both
coroutines defined using the `async` keyword.

In [19]:
import asyncio

@enact.typed_invokable(type(None), int)
@dataclasses.dataclass
class AsyncDieRoll(enact.AsyncInvokable):
  """Asynchronously rolls a die and returns the result."""
  dice_nr: int

  async def api_call(self) -> int:
    """Mimic calling into an API to obtain roll result."""
    print(f'Rolling die nr {self.dice_nr}.')
    await asyncio.sleep(random.random() * 0.1)
    result = random.randint(1, 6)
    print(f'Got result for die nr {self.dice_nr}: {result}')
    return result

  async def call(self) -> int:
    return await self.api_call()


@enact.typed_invokable(int, int)
class AsyncDiceRoll(enact.AsyncInvokable):
  """Asynchronously rolls the requested number of dice."""

  async def call(self, num_dice: int) -> int:
    """Roll the requested number of dice concurrently."""
    dice = [AsyncDieRoll(i) for i in range(num_dice)]
    rolls = [die() for die in dice]
    return sum(await asyncio.gather(*rolls))

with store:
  # Jupyter notebooks have their own event loop, so we can use await rather than
  # asyncio.run:
  invocation = await AsyncDiceRoll().invoke(num_rolls)
  print(f'Sum: {invocation.get_output()}')
  
  print(f'\nReplay last roll:')
  invocation = invocation.rewind()  # Rewind the last call.
  invocation = await invocation.replay_async()
  print(f'Sum: {invocation.get_output()}')

Rolling die nr 0.
Rolling die nr 1.
Rolling die nr 2.
Got result for die nr 1: 2
Got result for die nr 0: 5
Got result for die nr 2: 6
Sum: 13

Replay last roll:
Rolling die nr 2.
Got result for die nr 2: 3
Sum: 10


Python coroutines allow scheduling background tasks, which execute outside
of their calling context. Since enact needs to track the results of
subinvocations, this is not supported and will raise an error.

In [20]:
@enact.typed_invokable(int, int)
class WaitForFirstResult(enact.AsyncInvokable):
  """Rolls the requested number of dice and waits on the first result."""

  async def call(self, num_dice: int) -> int:
    """Roll the requested number of dice concurrently."""
    loop = asyncio.get_running_loop()
    # Create background tasks for each die roll.
    tasks = [loop.create_task(AsyncDieRoll(i)()) for i in range(num_dice)]
    # Wait for the first result.
    done, pending = await asyncio.wait(tasks, return_when=asyncio.FIRST_COMPLETED) 
    # Leave pending tasks running.
    return await done[0]

with store:
  try:
    invocation = await WaitForFirstResult().invoke(num_rolls)
  except enact.IncompleteSubinvocationError as e:
    print(f"Invocation failed: {e}")

Rolling die nr 0.
Rolling die nr 1.
Rolling die nr 2.
Got result for die nr 1: 6
Invocation failed: Subinvocation 0 did not complete during invocation of parent: AsyncDieRoll(dice_nr=0) invoked on None


#### Requesting and resolving multiple inputs.

Note that `async` can be used to request multiple inputs from external sources at once:

In [22]:
@enact.typed_invokable(type(None), int)
@dataclasses.dataclass
class AsyncUserRoll(enact.AsyncInvokable):
  roll_id: int
  async def call(self):
    # We provide the roll ID so that we can later distinguish between different
    # input requests.
    return enact.request_input(int, for_value=self.roll_id)

@enact.typed_invokable(type(None), int)
class CrowdSourcedDiceRoll(enact.AsyncInvokable):
  """Generates a batch of input requests for each dice roll."""
  async def call(self):
    rolls = [AsyncUserRoll(roll_id=i) for i in range(10)]
    # Wait for all rolls to complete
    results = await asyncio.gather(*[r() for r in rolls], return_exceptions=True)
    for r in results:
      if isinstance(r, Exception):
        raise r
    return sum(results)

with store:
  invocation = await CrowdSourcedDiceRoll().invoke()
  assert isinstance(invocation.get_raised(), enact.InputRequest)

  def provide_input(exception_ref):
    """Resolves an InputRequest exception by providing a dice roll."""
    input_request = exception_ref()
    assert isinstance(input_request, enact.InputRequest)
    roll_id = input_request.for_value()
    roll = (roll_id % 7) + 1
    print(f'injecting input {roll} for roll {input_request.for_value()}')
    return roll
  
  invocation = await invocation.replay_async(exception_override=provide_input)
  print(f'result: {invocation.get_output()}')


injecting input 1 for roll 0
injecting input 2 for roll 1
injecting input 3 for roll 2
injecting input 4 for roll 3
injecting input 5 for roll 4
injecting input 6 for roll 5
injecting input 7 for roll 6
injecting input 1 for roll 7
injecting input 2 for roll 8
injecting input 3 for roll 9
result: 34
